In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# 1. Load the dataset
data = pd.read_csv('C:/dataset/ae_data.csv')  # 파일 경로 수정
X = data.drop(columns=['y', 'time']).values  # Features only (excluding 'y' and 'time')
y = data['y'].values  # Labels

# 2. Normalize the data
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# Split data: normal (lying) and abnormal (jumping)
X_normal = X_scaled[y == 0]  # Normal (lying)
X_abnormal = X_scaled[y == 1]  # Abnormal (jumping)

# 3. Create a test dataset with mixed normal and abnormal data
X_test = np.vstack((X_normal, X_abnormal))
y_test = np.hstack((np.zeros(len(X_normal)), np.ones(len(X_abnormal))))  # 0: normal, 1: abnormal

# Shuffle test data
X_test, y_test = shuffle(X_test, y_test, random_state=42)

# 4. Train Autoencoder using normal data
# Autoencoder model
input_dim = X_normal.shape[1]

autoencoder = Sequential([
    Dense(32, activation='relu', input_dim=input_dim),
    Dense(16, activation='relu'),
    Dense(32, activation='relu'),
    Dense(input_dim, activation='sigmoid')  # Output layer matches input dimension
])

# Compile the autoencoder
autoencoder.compile(optimizer='adam', loss='mse')

# Train the autoencoder
autoencoder.fit(X_normal, X_normal, epochs=50, batch_size=32, validation_split=0.2, verbose=1)

# 5. Calculate reconstruction error on the test data
X_test_reconstructed = autoencoder.predict(X_test)
reconstruction_error = np.mean(np.square(X_test - X_test_reconstructed), axis=1)

# 6. Determine a threshold for classification
threshold = np.percentile(reconstruction_error, 95)  # Example: 95th percentile

# Classify based on the threshold
y_pred = (reconstruction_error > threshold).astype(int)

# 7. Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Threshold: {threshold}")
print(f"Accuracy: {accuracy:.4f}")
